schemes of partition: 
1. lomuto
2. hoare 
3. dijkstra (tripartite)


pivot element: 
1. last 
2. random 
3. median (last first middle)
4. median of 3 random elements
5. double pivot


metrics: 
1. exec_time 
2. comparisons 
3. swaps
4. memory_used


data: 
1. identical elements,
2. completely sorted,
3. random,
4. almost sorted,
5. reverse order, 
6. "triangular" (the first half is strictly ascending sequence, and the second half is a mirror image of the first), 
7. several different

In [194]:
import random
import time
import sys


In [195]:
class Metrics:
    def __init__(self):
        self.comparisons = 0
        self.swaps = 0
        self.memory_used = 0
        self.exec_time = 0

    def start_timer(self):
        self.start_time = time.time()

    def stop_timer(self):
        self.exec_time = time.time() - self.start_time
        
    def __str__(self):
        return f"Comparisons: {self.comparisons}, Swaps: {self.swaps}, Memory: {self.memory_used}, Time: {self.exec_time:.6f} sec"


In [196]:
class PartitionScheme:
    def lomuto(self, arr, low, high, pivot, metrics):
        i = low - 1
        for j in range(low, high):
            metrics.comparisons += 1
            if arr[j] <= pivot:
                i += 1
                metrics.comparisons += 1
                if i != j:
                    metrics.swaps += 1
                    arr[i], arr[j] = arr[j], arr[i]
        metrics.swaps += 1
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1

    def hoare(self, arr, low, high, pivot, metrics):
        i, j = low, high
        while True:
            while arr[i] < pivot:
                metrics.comparisons += 1
                i += 1
            metrics.comparisons += 1
            while arr[j] > pivot:
                metrics.comparisons += 1
                j -= 1
            metrics.comparisons += 1

            metrics.comparisons += 1
            if i >= j:
                return j
            
            metrics.swaps += 1
            arr[i], arr[j] = arr[j], arr[i]
            i += 1
            j -= 1

In [197]:
class PivotSelector:
    def last(self, arr, low, high, metrics):
        return arr[high]

    def random(self, arr, low, high, metrics):
        return arr[random.randint(low, high)]

    def median_of_three(self, arr, low, high, metrics):
        mid = (low + high) // 2
        metrics.comparisons += 1
        if arr[mid] < arr[low]:
            metrics.swaps += 1
            arr[mid], arr[low] = arr[low], arr[mid]
        metrics.comparisons += 1
        if arr[high] > arr[low]:
            metrics.swaps += 1
            arr[high], arr[low] = arr[low], arr[high]
        metrics.comparisons += 1
        if arr[high] < arr[mid]:
            metrics.swaps += 1
            arr[high], arr[mid] = arr[mid], arr[high]
        return arr[mid]

    def median_of_three_random(self, arr, low, high, metrics):
        indices = random.sample(range(low, high + 1), 3)
        a, b, c = indices[0], indices[1], indices[2]

        metrics.comparisons += 1
        if arr[b] < arr[a]:
            metrics.swaps += 1
            arr[a], arr[b] = arr[b], arr[a]
        metrics.comparisons += 1
        if arr[c] < arr[a]:
            metrics.swaps += 1
            arr[a], arr[c] = arr[c], arr[a]
        metrics.comparisons += 1
        if arr[c] < arr[b]:
            metrics.swaps += 1
            arr[b], arr[c] = arr[c], arr[b]

        return arr[b]

In [198]:
class DataGenerator:
    def identical(self, size, value = 1):
        return [value] * size

    def sorted(self, size):
        return list(range(size))

    def random(self, size, low=0, high=1000):
        return [random.randint(low, high) for _ in range(size)]

    def almost_sorted(self, size, swap_percentage=5):
        arr = list(range(size))
        swaps = max(1, (swap_percentage * size) // 100)
        print(swaps)
        for _ in range(swaps):
            i, j = random.sample(range(size), 2)
            arr[i], arr[j] = arr[j], arr[i]
        return arr

    def reverse_order(self, size):
        return list(range(size, 0, -1))

    def triangular(self, size):
        half = size // 2
        first_half = list(range(half))
        second_half = first_half[::-1]
        return first_half + second_half

In [ ]:

class SortRunner:
    def __init__(self):
        self.partition = PartitionScheme()
        self.pivot_selector = PivotSelector()
        self.metrics = Metrics()
        self.data_gen = DataGenerator()

    def quicksort(self, arr, low, high, partition_method, pivot_method, metrics):
        if low < high:
            pivot = pivot_method(arr, low, high, metrics)
            partition_point = partition_method(arr, low, high, pivot, metrics)
            self.quicksort(arr, low, partition_point - 1, partition_method, pivot_method, metrics)
            self.quicksort(arr, partition_point + 1, high, partition_method, pivot_method, metrics)
            
    def run(self, data_type, size, partition_method, pivot_method):
        data = data_type(size)
        self.metrics.start_timer()
        self.quicksort(data, 0, len(data) - 1, partition_method, pivot_method, self.metrics)
        self.metrics.stop_timer()
        return data, self.metrics


In [ ]:
runner = SortRunner()
data, metrics = runner.run(
    data_type=runner.data_gen.random,  
    size = 10,
    partition_method = runner.partition.lomuto,
    pivot_method = runner.pivot_selector.random
)

# Проверяем результат сортировки
assert data == sorted(data), f"Ошибка: массив отсортирован неправильно! Результат: {data}"

print("Отсортированные данные:", data)
print("Метрики:", metrics)

AssertionError: Ошибка: массив отсортирован неправильно! Результат: [49, 174, 83, 209, 345, 404, 582, 471, 748, 756]